In [9]:
import os
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

from utils_data import Vocabulary, Vectorizer, HeadQA, clean_words, parse_dataset, save_dataset_to_pickle, load_dataset_from_pickle 
from utils_data import random_oversamplig, mixed_oversampling, similarity_instance, translate_instance, translate_instance_ir, similarity_instance_ir


from training import train, validate, evaluate, evaluator, evaluate_better, get_optimizer, evaluate_better

from supervised_models import LogisticRegression

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from datasets import load_dataset

data_es = load_dataset('head_qa', 'es' )

Reusing dataset head_qa (C:\Users\tec005m\.cache\huggingface\datasets\head_qa\es\1.1.0\473dc5357942a3ff52963bd73cad0d167bd1bbc1ca5ca0732ee7372b480dd735)


In [11]:
training, validation, testing = data_es['train'], data_es['validation'], data_es['test']

In [12]:
for d in training:
    print(d)
    break;

{'answers': [{'aid': 1, 'atext': 'Son de tipo todo o nada.'}, {'aid': 2, 'atext': 'Son hiperpolarizantes.'}, {'aid': 3, 'atext': 'Se pueden sumar.'}, {'aid': 4, 'atext': 'Se propagan a largas distancias.'}, {'aid': 5, 'atext': 'Presentan un periodo refractario.'}], 'category': 'biology', 'image': '', 'name': 'Cuaderno_2013_1_B', 'qid': 1, 'qtext': 'Los potenciales postsinápticos excitadores:', 'ra': 3, 'year': '2013'}


In [13]:
# training_instances = parse_dataset(training)
# validation_instances = parse_dataset(validation)
# testing_instances = parse_dataset(testing)

# oversampled_training = random_oversamplig(training_instances)
# mixed_oversampling_training = mixed_oversampling(training_instances, translate_instance, similarity_instance)

# save_dataset_to_pickle('../data/training.pickle', training_instances)
# save_dataset_to_pickle('../data/validation.pickle', validation_instances)
# save_dataset_to_pickle('../data/testing.pickle', testing_instances)
# save_dataset_to_pickle('../data/oversampled_training.pickle', oversampled_training)

In [14]:
training_instances_ir = load_dataset_from_pickle('../data/training_ir.pickle')

In [15]:
training_instances = load_dataset_from_pickle('../data/training.pickle')
validation_instances = load_dataset_from_pickle('../data/validation.pickle')
testing_instances = load_dataset_from_pickle('../data/testing.pickle')
oversampled_training = load_dataset_from_pickle('../data/oversampled_training.pickle')

In [16]:
mixed_oversampling_training = mixed_oversampling(training_instances, translate_instance, similarity_instance)

0/2657
1/2657
2/2657
3/2657
4/2657
5/2657
6/2657
7/2657
8/2657
9/2657
10/2657
11/2657
12/2657
13/2657
14/2657
15/2657
16/2657
17/2657
18/2657
19/2657
20/2657
21/2657
22/2657
23/2657
24/2657
25/2657
26/2657
27/2657
28/2657
29/2657
30/2657
31/2657
32/2657
33/2657
34/2657
35/2657
36/2657
37/2657
38/2657
39/2657
40/2657
41/2657
42/2657
43/2657
44/2657
45/2657
46/2657
47/2657
48/2657
49/2657
50/2657
51/2657
52/2657
53/2657
54/2657
55/2657
56/2657
57/2657
58/2657
59/2657
60/2657
61/2657
62/2657
63/2657
64/2657
65/2657
66/2657
67/2657
68/2657
69/2657
70/2657
71/2657
72/2657
73/2657
74/2657
75/2657
76/2657
77/2657
78/2657
79/2657
80/2657
81/2657
82/2657
83/2657
84/2657
85/2657
86/2657
87/2657
88/2657
89/2657
90/2657
91/2657
92/2657
93/2657
94/2657
95/2657
96/2657
97/2657
98/2657
99/2657
100/2657
101/2657
102/2657
103/2657
104/2657
105/2657
106/2657
107/2657
108/2657
109/2657
110/2657
111/2657
112/2657
113/2657
114/2657
115/2657
116/2657
117/2657
118/2657
119/2657
120/2657
121/2657
122/2657
123

923/2657
924/2657
925/2657
926/2657
927/2657
928/2657
929/2657
930/2657
931/2657
932/2657
933/2657
934/2657
935/2657
936/2657
937/2657
938/2657
939/2657
940/2657
941/2657
942/2657
943/2657
944/2657
945/2657
946/2657
947/2657
948/2657
949/2657
950/2657
951/2657
952/2657
953/2657
954/2657
955/2657
956/2657
957/2657
958/2657
959/2657
960/2657
961/2657
962/2657
963/2657
964/2657
965/2657
966/2657
967/2657
968/2657
969/2657
970/2657
971/2657
972/2657
973/2657
974/2657
975/2657
976/2657
977/2657
978/2657
979/2657
980/2657
981/2657
982/2657
983/2657
984/2657
985/2657
986/2657
987/2657
988/2657
989/2657
990/2657
991/2657
992/2657
993/2657
994/2657
995/2657
996/2657
997/2657
998/2657
999/2657
1000/2657
1001/2657
1002/2657
1003/2657
1004/2657
1005/2657
1006/2657
1007/2657
1008/2657
1009/2657
1010/2657
1011/2657
1012/2657
1013/2657
1014/2657
1015/2657
1016/2657
1017/2657
1018/2657
1019/2657
1020/2657
1021/2657
1022/2657
1023/2657
1024/2657
1025/2657
1026/2657
1027/2657
1028/2657
1029/2657
1030/26

1750/2657
1751/2657
1752/2657
1753/2657
1754/2657
1755/2657
1756/2657
1757/2657
1758/2657
1759/2657
1760/2657
1761/2657
1762/2657
1763/2657
1764/2657
1765/2657
1766/2657
1767/2657
1768/2657
1769/2657
1770/2657
1771/2657
1772/2657
1773/2657
1774/2657
1775/2657
1776/2657
1777/2657
1778/2657
1779/2657
1780/2657
1781/2657
1782/2657
1783/2657
1784/2657
1785/2657
1786/2657
1787/2657
1788/2657
1789/2657
1790/2657
1791/2657
1792/2657
1793/2657
1794/2657
1795/2657
1796/2657
1797/2657
1798/2657
1799/2657
1800/2657
1801/2657
1802/2657
1803/2657
1804/2657
1805/2657
1806/2657
1807/2657
1808/2657
1809/2657
1810/2657
1811/2657
1812/2657
1813/2657
1814/2657
1815/2657
1816/2657
1817/2657
1818/2657
1819/2657
1820/2657
1821/2657
1822/2657
1823/2657
1824/2657
1825/2657
1826/2657
1827/2657
1828/2657
1829/2657
1830/2657
1831/2657
1832/2657
1833/2657
1834/2657
1835/2657
1836/2657
1837/2657
1838/2657
1839/2657
1840/2657
1841/2657
1842/2657
1843/2657
1844/2657
1845/2657
1846/2657
1847/2657
1848/2657
1849/2657


2570/2657
2571/2657
2572/2657
2573/2657
2574/2657
2575/2657
2576/2657
2577/2657
2578/2657
2579/2657
2580/2657
2581/2657
2582/2657
2583/2657
2584/2657
2585/2657
2586/2657
2587/2657
2588/2657
2589/2657
2590/2657
2591/2657
2592/2657
2593/2657
2594/2657
2595/2657
2596/2657
2597/2657
2598/2657
2599/2657
2600/2657
2601/2657
2602/2657
2603/2657
2604/2657
2605/2657
2606/2657
2607/2657
2608/2657
2609/2657
2610/2657
2611/2657
2612/2657
2613/2657
2614/2657
2615/2657
2616/2657
2617/2657
2618/2657
2619/2657
2620/2657
2621/2657
2622/2657
2623/2657
2624/2657
2625/2657
2626/2657
2627/2657
2628/2657
2629/2657
2630/2657
2631/2657
2632/2657
2633/2657
2634/2657
2635/2657
2636/2657
2637/2657
2638/2657
2639/2657
2640/2657
2641/2657
2642/2657
2643/2657
2644/2657
2645/2657
2646/2657
2647/2657
2648/2657
2649/2657
2650/2657
2651/2657
2652/2657
2653/2657
2654/2657
2655/2657
2656/2657


In [17]:
save_dataset_to_pickle('../data/mixed_oversampling_training.pickle', mixed_oversampling_training)

In [18]:
mixed_oversampling_training_ir = mixed_oversampling(training_instances_ir, translate_instance_ir, similarity_instance_ir)

0/2657
1/2657
2/2657
3/2657
4/2657
5/2657
6/2657
7/2657
8/2657
9/2657
10/2657
11/2657
12/2657
13/2657
14/2657
15/2657
16/2657
17/2657
18/2657
19/2657
20/2657
21/2657
22/2657
23/2657
24/2657
25/2657
26/2657
27/2657
28/2657
29/2657
30/2657
31/2657
32/2657
33/2657
34/2657
35/2657
36/2657
37/2657
38/2657
39/2657
40/2657
41/2657
42/2657
43/2657
44/2657
45/2657
46/2657
47/2657
48/2657
49/2657
50/2657
51/2657
52/2657
53/2657
54/2657
55/2657
56/2657
57/2657
58/2657
59/2657
60/2657
61/2657
62/2657
63/2657
64/2657
65/2657
66/2657
67/2657
68/2657
69/2657
70/2657
71/2657
72/2657
73/2657
74/2657
75/2657
76/2657
77/2657
78/2657
79/2657
80/2657
81/2657
82/2657
83/2657
84/2657
85/2657
86/2657
87/2657
88/2657
89/2657
90/2657
91/2657
92/2657
93/2657
94/2657
95/2657
96/2657
97/2657
98/2657
99/2657
100/2657
101/2657
102/2657
103/2657
104/2657
105/2657
106/2657
107/2657
108/2657
109/2657
110/2657
111/2657
112/2657
113/2657
114/2657
115/2657
116/2657
117/2657
118/2657
119/2657
120/2657
121/2657
122/2657
123

923/2657
924/2657
925/2657
926/2657
927/2657
928/2657
929/2657
930/2657
931/2657
932/2657
933/2657
934/2657
935/2657
936/2657
937/2657
938/2657
939/2657
940/2657
941/2657
942/2657
943/2657
944/2657
945/2657
946/2657
947/2657
948/2657
949/2657
950/2657
951/2657
952/2657
953/2657
954/2657
955/2657
956/2657
957/2657
958/2657
959/2657
960/2657
961/2657
962/2657
963/2657
964/2657
965/2657
966/2657
967/2657
968/2657
969/2657
970/2657
971/2657
972/2657
973/2657
974/2657
975/2657
976/2657
977/2657
978/2657
979/2657
980/2657
981/2657
982/2657
983/2657
984/2657
985/2657
986/2657
987/2657
988/2657
989/2657
990/2657
991/2657
992/2657
993/2657
994/2657
995/2657
996/2657
997/2657
998/2657
999/2657
1000/2657
1001/2657
1002/2657
1003/2657
1004/2657
1005/2657
1006/2657
1007/2657
1008/2657
1009/2657
1010/2657
1011/2657
1012/2657
1013/2657
1014/2657
1015/2657
1016/2657
1017/2657
1018/2657
1019/2657
1020/2657
1021/2657
1022/2657
1023/2657
1024/2657
1025/2657
1026/2657
1027/2657
1028/2657
1029/2657
1030/26

1750/2657
1751/2657
1752/2657
1753/2657
1754/2657
1755/2657
1756/2657
1757/2657
1758/2657
1759/2657
1760/2657
1761/2657
1762/2657
1763/2657
1764/2657
1765/2657
1766/2657
1767/2657
1768/2657
1769/2657
1770/2657
1771/2657
1772/2657
1773/2657
1774/2657
1775/2657
1776/2657
1777/2657
1778/2657
1779/2657
1780/2657
1781/2657
1782/2657
1783/2657
1784/2657
1785/2657
1786/2657
1787/2657
1788/2657
1789/2657
1790/2657
1791/2657
1792/2657
1793/2657
1794/2657
1795/2657
1796/2657
1797/2657
1798/2657
1799/2657
1800/2657
1801/2657
1802/2657
1803/2657
1804/2657
1805/2657
1806/2657
1807/2657
1808/2657
1809/2657
1810/2657
1811/2657
1812/2657
1813/2657
1814/2657
1815/2657
1816/2657
1817/2657
1818/2657
1819/2657
1820/2657
1821/2657
1822/2657
1823/2657
1824/2657
1825/2657
1826/2657
1827/2657
1828/2657
1829/2657
1830/2657
1831/2657
1832/2657
1833/2657
1834/2657
1835/2657
1836/2657
1837/2657
1838/2657
1839/2657
1840/2657
1841/2657
1842/2657
1843/2657
1844/2657
1845/2657
1846/2657
1847/2657
1848/2657
1849/2657


2570/2657
2571/2657
2572/2657
2573/2657
2574/2657
2575/2657
2576/2657
2577/2657
2578/2657
2579/2657
2580/2657
2581/2657
2582/2657
2583/2657
2584/2657
2585/2657
2586/2657
2587/2657
2588/2657
2589/2657
2590/2657
2591/2657
2592/2657
2593/2657
2594/2657
2595/2657
2596/2657
2597/2657
2598/2657
2599/2657
2600/2657
2601/2657
2602/2657
2603/2657
2604/2657
2605/2657
2606/2657
2607/2657
2608/2657
2609/2657
2610/2657
2611/2657
2612/2657
2613/2657
2614/2657
2615/2657
2616/2657
2617/2657
2618/2657
2619/2657
2620/2657
2621/2657
2622/2657
2623/2657
2624/2657
2625/2657
2626/2657
2627/2657
2628/2657
2629/2657
2630/2657
2631/2657
2632/2657
2633/2657
2634/2657
2635/2657
2636/2657
2637/2657
2638/2657
2639/2657
2640/2657
2641/2657
2642/2657
2643/2657
2644/2657
2645/2657
2646/2657
2647/2657
2648/2657
2649/2657
2650/2657
2651/2657
2652/2657
2653/2657
2654/2657
2655/2657
2656/2657


In [19]:
save_dataset_to_pickle('../data/mixed_oversampling_training_ir.pickle', mixed_oversampling_training_ir)

In [23]:
a = load_dataset_from_pickle('../data/mixed_oversampling_training.pickle')

In [26]:
len(a)

18599

In [ ]:
vectorizer = Vectorizer.vectorize_training(oversampled_training)

In [ ]:
trainset = HeadQA(instances=training_instances, vectorizer=vectorizer, right_padding=False, max_length=30)
validset = HeadQA(instances=validation_instances, vectorizer=vectorizer, right_padding=False, max_length=30)
testset = HeadQA(instances=testing_instances, vectorizer=vectorizer, right_padding=False, max_length=30)

In [ ]:
batch_size = 32
train_dt = DataLoader(trainset, batch_size=batch_size,shuffle=True, drop_last=True)
valid_dt = DataLoader(validset, batch_size=batch_size,shuffle=True, drop_last=True)
test_dt = DataLoader(testset, batch_size=batch_size,shuffle=True, drop_last=True)

In [ ]:
model = LogisticRegression(trainset.max_length, 1)
optimizer = get_optimizer(model, lr = 0.01, wd = 1e-5)

In [ ]:
training_results = train(model, optimizer, train_dt, valid_dt, validate, epochs=20)

In [ ]:
acc, points = evaluate(model, validation, trainset.encode, evaluator)

In [ ]:
acc, points

In [ ]:
acc, points = evaluate(model, testing, trainset.encode, evaluator)

In [ ]:
acc, points

In [ ]:
# model_path = os.getcwd() + '/trained_models/logistic_regressor'
# torch.save(model.state_dict(), model_path)

In [ ]:
save_dataset_to_pickle('../data/train_results_lreg_sigmoid.pickle', training_results)

In [ ]:
training_results = load_dataset_from_pickle('../data/train_results_lreg.pickle')

In [ ]:
model = LogisticRegression(trainset.max_length, 1)
model.load_state_dict(torch.load(os.getcwd() + '/trained_models/logistic_regressor'))
model.eval()

In [ ]:
acc, points, acc_list, points_list = evaluate_better(model, validation, trainset.encode, evaluator)
acc, points, acc_list, points_list

In [ ]:
acc, points, acc_list, points_list = evaluate_better(model, validation, trainset.encode, evaluator)
acc, points, acc_list, points_list